In [165]:
import nltk
from nltk.tokenize import treebank
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import math
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random
from nltk.corpus import opinion_lexicon
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 

In [166]:
lemmatizer = WordNetLemmatizer() 

In [167]:
dataset = pd.read_csv('deceptive-opinion.csv')
stop_words = set(stopwords.words('english'))

In [168]:
lemmatizer.lemmatize("was'nt") in stop_words

False

In [169]:
def train_test_split(df,test_size):
    if isinstance(test_size,float):
        test_size = round(test_size * len(df))
        
    if(test_size > len(df)):
        return 0, df

    indices = df.index.tolist()
    test_indices = random.sample(population=indices,k = test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df
train_df,test_df = train_test_split(dataset,0.3)

In [170]:
def vaderSentimentValue(review):
    sid_obj =  SentimentIntensityAnalyzer()
    
    sentiment_dict = sid_obj.polarity_scores(review)
    return sentiment_dict['compound']

In [171]:
positive_words_list = set(opinion_lexicon.positive())
negative_words_list = set(opinion_lexicon.negative())
tokenizer = treebank.TreebankWordTokenizer()
def getSentimentValue(review):
    senti = 0
    words = [word.lower() for word in tokenizer.tokenize(review)]
    for word in words:
        if word in positive_words_list:
            senti += 1
        elif word in negative_words_list:
            senti -= 1
    return senti

In [172]:
fake_vocab = {}
notfake_vocab = {}
stop_words = stopwords.words('english')

In [173]:
(wn.synsets("bad")[0]).lemmas()[1]
    

Lemma('bad.n.01.badness')

In [123]:
fake_vocab = {}
notfake_vocab = {}
for i in range(len(train_df)):
    review = list(train_df['text'])[i]
    words = word_tokenize(review)
    for word in words:
        if word.lower() not in stop_words and len(word) != 1:
            if list(train_df['deceptive'])[i] == "truthful":
                if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                    notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                else:
                    notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
            elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
            else:
                fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1


fake_vocab = sorted(fake_vocab.items(), key = 
         lambda kv:(-kv[1], kv[0]))   
notfake_vocab =sorted(notfake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))  
finalFakeVocab = []
finalNotFakeVocab = []
for i in range(150):
    finalFakeVocab.append(fake_vocab[i][0])
    finalNotFakeVocab.append(notfake_vocab[i][0])
fake_vocab = finalFakeVocab
notfake_vocab = finalNotFakeVocab

old_len_fakevocab = len(fake_vocab)
old_len_notfakevocab = len(notfake_vocab)
new_len_fakevocab = 0;
new_len_notfakevocab = 0;

while(True):
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    for word in notfake_vocab:
        for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                            for lemma in synset.lemmas():
                                if lemma.name() not in notfake_vocab:
                                    notfake_vocab.append(lemma.name())



    for word in fake_vocab:
        for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                            for lemma in synset.lemmas():
                                if lemma.name() not in fake_vocab:
                                    fake_vocab.append(lemma.name())

    new_len_fakevocab = len(fake_vocab);
    new_len_notfakevocab = len(notfake_vocab);

    if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
        break

In [124]:
notfake_vocab

['room',
 'hotel',
 'stay',
 "n't",
 'great',
 'would',
 'chicago',
 'staff',
 'night',
 'one',
 'service',
 'bed',
 'location',
 'stayed',
 'u',
 'get',
 "'s",
 'time',
 '...',
 'nice',
 'could',
 'desk',
 'good',
 'floor',
 'bathroom',
 'even',
 'day',
 'also',
 'clean',
 'view',
 'like',
 'two',
 'front',
 'got',
 'back',
 'restaurant',
 'breakfast',
 'lobby',
 'friendly',
 'go',
 'place',
 'small',
 'comfortable',
 'well',
 'door',
 'really',
 'bar',
 'first',
 'never',
 'free',
 'called',
 'michigan',
 'area',
 'helpful',
 'booked',
 'price',
 'told',
 'next',
 'better',
 'minute',
 'recommend',
 'guest',
 'morning',
 'experience',
 'made',
 'suite',
 'business',
 'elevator',
 'much',
 '--',
 'check',
 'reservation',
 'right',
 'take',
 'best',
 'asked',
 'city',
 'call',
 'thing',
 'everything',
 'many',
 'people',
 'problem',
 'trip',
 'review',
 'coffee',
 'rate',
 'staying',
 'water',
 'every',
 'excellent',
 'said',
 'say',
 'walk',
 'weekend',
 'another',
 'close',
 'large',

In [125]:
def getFakenessValue(review):
    senti = 0
    words = [word.lower() for word in tokenizer.tokenize(review)]
    for word in words:
        if lemmatizer.lemmatize(word) in notfake_vocab:
            senti += 1
        if lemmatizer.lemmatize(word) in fake_vocab:
            senti -= 1
    return senti

In [127]:
test_df['fakescore'] = test_df['text'].apply(getFakenessValue)

In [ ]:
test_df['vader_sent_score'] = test_df['text'].apply(vaderSentimentValue)
test_df['sentiment_score'] = test_df['text'].apply(getSentimentValue)

In [128]:
test_df

,deceptive,hotel,polarity,source,text,fakescore,vader_sent_score,sentiment_score,true_label,index,predicted_label
1220,deceptive,hyatt,negative,MTurk,Hyatt Regency Chicago seemed like a nice place...,-1,-0.8073,-3,1,0,-1
1200,deceptive,fairmont,negative,MTurk,My wife and I live in the Western Suburbs of C...,-1,0.9416,7,1,1,-1
1139,truthful,allegro,negative,Web,This hotel must have originally been an ordina...,0,0.7978,1,-1,2,-1
353,truthful,intercontinental,positive,TripAdvisor,Stayed at the InterContinental for an entire w...,2,0.8791,9,-1,3,-1
1103,truthful,james,negative,Web,Very disapointed in the service and quality of...,0,-0.2960,-1,-1,4,-1
...,...,...,...,...,...,...,...,...,...,...,...
29,truthful,hyatt,positive,TripAdvisor,I booked a room at the Hyatt through Priceline...,-1,0.9687,5,-1,475,-1
949,truthful,homewood,negative,Web,This hotel is trading on the Hilton name and a...,0,-0.8360,-2,-1,476,-1
546,deceptive,knickerbocker,positive,MTurk,This hotel was worth every cent. You have not ...,-1,0.8243,3,1,477,-1
398,truthful,amalfi,positive,TripAdvisor,This hotel was wonderful! I am a college stude...,2,0.9752,4,-1,478,-1


In [129]:
P1 = 54
P2 = 1393 
#for these 2 values the below two classifiers gave 94 and 73% accuracy respectively
#1203,76   1161,150 (1163, 1399) (662, 1476) (1542, 194) (78, 28) (699, 44) (1384, 192)

In [130]:
while(True):
    p1 = random.randrange(len(test_df))
    p2 = random.randrange(len(test_df))
    if(p1 != p2):
        break

In [131]:
# p1 = P1
# p2=P2
p1,p2

(292, 229)

In [132]:

point12 = list(test_df['fakescore'])[p1]

point22 = list(test_df['fakescore'])[p2]

def Distance(x1,y1,x2,y2):
    return (x1-x2)**2 + (y1-y2)**2
    

In [133]:

cluster12 = {p1:point12}

cluster22 = {p2:point22}

old_centroid12 = new_centroid12 = point12

old_centroid22 = new_centroid22 = point22


while(True):
   # old_centroid11 = new_centroid11
    old_centroid12 = new_centroid12
    #old_centroid21 = new_centroid21
    old_centroid22 = new_centroid22
    for i in test_df.index:
       # point1 = test_df['vader_sent_score'][i]
        point2 = test_df['fakescore'][i]
        
        if(Distance(0,old_centroid12,0,point2) > Distance(0,old_centroid22,0,point2)):
            if(i not in cluster22):
               # cluster21[i] = point1
                cluster22[i] = point2
        else:
            if(i not in cluster12):
               # cluster11[i] = point1
                cluster12[i] = point2
        
        #new_centroid11 = sum(cluster11.values())/len(cluster11)
        new_centroid12 = sum(cluster12.values())/len(cluster12)
       # new_centroid21 = sum(cluster21.values())/len(cluster21)
        new_centroid22 = sum(cluster22.values())/len(cluster22)
    if( old_centroid12 == new_centroid12 and old_centroid22 == new_centroid22):
        break
        

In [134]:
if(Distance(0,old_centroid12,0,0) < Distance(0,old_centroid22,0,0)):
    label1 = 1 #fake
    label2 = -1 #real
else:
    label1 = -1
    label2 = 1





In [135]:
def getGivenLabel(value):
    if( value == "truthful"):
        return -1 #notfake
    else:
        return 1 #fake

test_df['true_label'] = test_df['deceptive'].apply(getGivenLabel)

test_df['index'] = list(range(len(test_df)))

def getPredictedLabel(value):  
    if(value in cluster11):
        return label1
    else:
        return label2
    
test_df['predicted_label'] = test_df['index'].apply(getPredictedLabel)

In [136]:
count = 0
for i in test_df.index:
    if(test_df['true_label'][i] == test_df['predicted_label'][i]):
        count += 1
            
count/len(test_df)

0.48541666666666666

In [137]:
truePositive = trueNegative = falsePositive = falseNegative = 0

In [138]:
for i in test_df.index:
    if(test_df['true_label'][i] == test_df['predicted_label'][i] == 1):
        truePositive += 1
    elif test_df['true_label'][i] == test_df['predicted_label'][i] == -1:
        trueNegative += 1
    elif test_df['true_label'][i] == -1 and test_df['predicted_label'][i] == 1:
        falsePositive += 1
    elif test_df['true_label'][i] == 1 and test_df['predicted_label'][i] == -1 :
        falseNegative += 1

In [139]:
accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
precision = truePositive/(truePositive + falsePositive)
recall =truePositive/(truePositive + falseNegative) 

accuracy,precision,recall

(0.48541666666666666, 0.5027027027027027, 0.7469879518072289)

In [140]:
truePositive = trueNegative = falsePositive = falseNegative = 0
for i in test_df.index:
    if(test_df['true_label'][i] == 1 and test_df['fakescore'][i] < 0):
        truePositive += 1
    elif test_df['true_label'][i] == -1 and test_df['fakescore'][i] > 0:
        trueNegative += 1
    elif test_df['true_label'][i] == -1 and test_df['fakescore'][i] < 0:
        falsePositive += 1
    elif test_df['true_label'][i] == 1 and test_df['fakescore'][i] > 0 :
        falseNegative += 1
        
accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
precision = truePositive/(truePositive + falsePositive)
recall =truePositive/(truePositive + falseNegative) 

accuracy,precision,recall

(0.7091412742382271, 0.696969696969697, 0.7540983606557377)

In [141]:
while(True):
    p1 = random.randrange(len(test_df))
    p2 = random.randrange(len(test_df))
    if(p1 != p2):
        break
point1 = list(test_df['fakescore'])[p1]

point2 = list(test_df['fakescore'])[p2]

def Distance(x1,x2):
    return (x1-x2)**2
    

In [142]:
cluster1 = {p1:point1}

cluster2 = {p2:point2}

old_centroid1 = new_centroid1 = point1

old_centroid2 = new_centroid2 = point2



while(True):
    old_centroid1 = new_centroid1

    old_centroid2 = new_centroid2
    
    for i in range(len(test_df)):
       
        point = list(test_df['fakescore'])[i]
        
        if(Distance(old_centroid1,point) > Distance(old_centroid2,point)):
            if(i not in cluster2):
                cluster2[i] = point
        else:
            if(i not in cluster11):
                cluster1[i] = point
               
        
        new_centroid1 = sum(cluster1.values())/len(cluster1)
        
        new_centroid2 = sum(cluster2.values())/len(cluster2)
    if(old_centroid1 == new_centroid1 and old_centroid2 == new_centroid2):
        break
        

In [143]:
if(Distance(old_centroid1,0) < Distance(old_centroid2,0)):
    label1 = -1
    label2 = 1
else:
    label1 = 1
    label2 = -1





In [144]:

def getPredictedLabel(value):  
    if(value in cluster1):
        return label1
    else:
        return label2
    
test_df['predicted_label'] = test_df['index'].apply(getPredictedLabel)

In [145]:
truePositive = trueNegative = falsePositive = falseNegative = 0
for i in test_df.index:
    if(test_df['true_label'][i] == test_df['predicted_label'][i] == 1):
        truePositive += 1
    elif test_df['true_label'][i] == test_df['predicted_label'][i] == -1:
        trueNegative += 1
    elif test_df['true_label'][i] == -1 and test_df['predicted_label'][i] == 1:
        falsePositive += 1
    elif test_df['true_label'][i] == 1 and test_df['predicted_label'][i] == -1 :
        falseNegative += 1

accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
precision = truePositive/(truePositive + falsePositive)
recall =truePositive/(truePositive + falseNegative) 

accuracy,precision,recall

(0.6354166666666666, 0.6434108527131783, 0.6666666666666666)

In [146]:
#for p1=274 and p2=453 the accuracy given by this is 85%

In [177]:
fake_vocab = {}
notfake_vocab = {}
for i in range(len(train_df)):
    review = list(train_df['text'])[i]
    words = word_tokenize(review)
    for word in words:
        if word.lower() not in stop_words and len(word) != 1:
            if list(train_df['deceptive'])[i] == "truthful":
                if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                    notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                else:
                    notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
            elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
            else:
                fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1


fake_vocab = sorted(fake_vocab.items(), key = 
         lambda kv:(-kv[1], kv[0]))   
notfake_vocab =sorted(notfake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))  
finalFakeVocab = []
finalNotFakeVocab = []
for i in range(200):
    finalFakeVocab.append(fake_vocab[i][0])
    finalNotFakeVocab.append(notfake_vocab[i][0])
fake_vocab = finalFakeVocab
notfake_vocab = finalNotFakeVocab

old_len_fakevocab = len(fake_vocab)
old_len_notfakevocab = len(notfake_vocab)
new_len_fakevocab = 0;
new_len_notfakevocab = 0;

while(True):
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    for word in notfake_vocab:
        for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                            for lemma in synset.lemmas():
                                if lemma.name() not in notfake_vocab:
                                    notfake_vocab.append(lemma.name())



    for word in fake_vocab:
        for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                            for lemma in synset.lemmas():
                                if lemma.name() not in fake_vocab:
                                    fake_vocab.append(lemma.name())

    new_len_fakevocab = len(fake_vocab);
    new_len_notfakevocab = len(notfake_vocab);

    if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
        break
        
test_df['fakescore'] = test_df['text'].apply(getFakenessValue)

In [178]:
while(True):
    p1 = random.randrange(len(test_df))
    p2 = random.randrange(len(test_df))
    if(p1 != p2):
        break
point1 = list(test_df['fakescore'])[p1]

point2 = list(test_df['fakescore'])[p2]

def Distance(x1,x2):
    return (x1-x2)**2
    

In [179]:
cluster1 = {p1:point1}

cluster2 = {p2:point2}

old_centroid1 = new_centroid1 = point1

old_centroid2 = new_centroid2 = point2



while(True):
    old_centroid1 = new_centroid1

    old_centroid2 = new_centroid2
    
    for i in range(len(test_df)):
       
        point = list(test_df['fakescore'])[i]
        
        if(Distance(old_centroid1,point) > Distance(old_centroid2,point)):
            if(i not in cluster2):
                cluster2[i] = point
        else:
            if(i not in cluster11):
                cluster1[i] = point
               
        
        new_centroid1 = sum(cluster1.values())/len(cluster1)
        
        new_centroid2 = sum(cluster2.values())/len(cluster2)
    if(old_centroid1 == new_centroid1 and old_centroid2 == new_centroid2):
        break
        

In [180]:
if(Distance(old_centroid1,0) < Distance(old_centroid2,0)):
    label1 = -1
    label2 = 1
else:
    label1 = 1
    label2 = -1





In [182]:
def getGivenLabel(value):
    if( value == "truthful"):
        return -1 #notfake
    else:
        return 1 #fake

test_df['true_label'] = test_df['deceptive'].apply(getGivenLabel)

test_df['index'] = list(range(len(test_df)))





def getPredictedLabel(value):  
    if(value in cluster1):
        return label1
    else:
        return label2
    
test_df['predicted_label'] = test_df['index'].apply(getPredictedLabel)

In [183]:
truePositive = trueNegative = falsePositive = falseNegative = 0
for i in test_df.index:
    if(test_df['true_label'][i] == test_df['predicted_label'][i] == 1):
        truePositive += 1
    elif test_df['true_label'][i] == test_df['predicted_label'][i] == -1:
        trueNegative += 1
    elif test_df['true_label'][i] == -1 and test_df['predicted_label'][i] == 1:
        falsePositive += 1
    elif test_df['true_label'][i] == 1 and test_df['predicted_label'][i] == -1 :
        falseNegative += 1

accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
precision = truePositive/(truePositive + falsePositive)
recall =truePositive/(truePositive + falseNegative) 

accuracy,precision,recall

(0.6479166666666667, 0.6272401433691757, 0.7291666666666666)

In [197]:
fake_vocab = {}
notfake_vocab = {}
for i in range(len(train_df)):
    review = list(train_df['text'])[i]
    words = word_tokenize(review)
    for word in words:
        if word.lower() not in stop_words and len(word) != 1:
            if list(train_df['deceptive'])[i] == "truthful":
                if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                    notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                else:
                    notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
            elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
            else:
                fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1


fake_vocab = sorted(fake_vocab.items(), key = 
         lambda kv:(-kv[1], kv[0]))   
notfake_vocab =sorted(notfake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))  
finalFakeVocab = []
finalNotFakeVocab = []
for i in range(500):
    finalFakeVocab.append(fake_vocab[i][0])
    finalNotFakeVocab.append(notfake_vocab[i][0])
fake_vocab = finalFakeVocab
notfake_vocab = finalNotFakeVocab

old_len_fakevocab = len(fake_vocab)
old_len_notfakevocab = len(notfake_vocab)
new_len_fakevocab = 0;
new_len_notfakevocab = 0;

while(True):
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    for word in notfake_vocab:
        for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                            for lemma in synset.lemmas():
                                if lemma.name() not in notfake_vocab:
                                    notfake_vocab.append(lemma.name())



    for word in fake_vocab:
        for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                            for lemma in synset.lemmas():
                                if lemma.name() not in fake_vocab:
                                    fake_vocab.append(lemma.name())

    new_len_fakevocab = len(fake_vocab);
    new_len_notfakevocab = len(notfake_vocab);

    if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
        break
        
test_df['fakescore'] = test_df['text'].apply(getFakenessValue)

In [198]:
while(True):
    p1 = random.randrange(len(test_df))
    p2 = random.randrange(len(test_df))
    if(p1 != p2):
        break
point1 = list(test_df['fakescore'])[p1]

point2 = list(test_df['fakescore'])[p2]

def Distance(x1,x2):
    return (x1-x2)**2
    

In [199]:
cluster1 = {p1:point1}

cluster2 = {p2:point2}

old_centroid1 = new_centroid1 = point1

old_centroid2 = new_centroid2 = point2



while(True):
    old_centroid1 = new_centroid1

    old_centroid2 = new_centroid2
    
    for i in range(len(test_df)):
       
        point = list(test_df['fakescore'])[i]
        
        if(Distance(old_centroid1,point) > Distance(old_centroid2,point)):
            if(i not in cluster2):
                cluster2[i] = point
        else:
            if(i not in cluster11):
                cluster1[i] = point
               
        
        new_centroid1 = sum(cluster1.values())/len(cluster1)
        
        new_centroid2 = sum(cluster2.values())/len(cluster2)
    if(old_centroid1 == new_centroid1 and old_centroid2 == new_centroid2):
        break
        

In [200]:
def getGivenLabel(value):
    if( value == "truthful"):
        return -1 #notfake
    else:
        return 1 #fake

test_df['true_label'] = test_df['deceptive'].apply(getGivenLabel)

test_df['index'] = list(range(len(test_df)))





def getPredictedLabel(value):  
    if(value in cluster1):
        return label1
    else:
        return label2
    
test_df['predicted_label'] = test_df['index'].apply(getPredictedLabel)

In [201]:
truePositive = trueNegative = falsePositive = falseNegative = 0
for i in test_df.index:
    if(test_df['true_label'][i] == test_df['predicted_label'][i] == 1):
        truePositive += 1
    elif test_df['true_label'][i] == test_df['predicted_label'][i] == -1:
        trueNegative += 1
    elif test_df['true_label'][i] == -1 and test_df['predicted_label'][i] == 1:
        falsePositive += 1
    elif test_df['true_label'][i] == 1 and test_df['predicted_label'][i] == -1 :
        falseNegative += 1

accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
precision = truePositive/(truePositive + falsePositive)
recall =truePositive/(truePositive + falseNegative) 

accuracy,precision,recall

(0.5958333333333333, 0.6236559139784946, 0.48333333333333334)